In [1]:
import datetime
from pyspark.sql import SparkSession
#or use .master("local[1]") instead of .master("spark://192.168.1.153:7077")
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.1.153:7077") \
        .appName("team8_03")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",1)\
        .getOrCreate()
        
sc = spark_session.sparkContext

In [2]:
def separate(x):
    for c in x:
        return (list(c))

def map_char(line):
    for x in line:
        return (x, 1)

In [3]:
def GC_content_count(path, filename):
    start = datetime.datetime.now()
    microbiome = sc.textFile(path)
    microbiome_lines = microbiome.map(lambda x: x.split('\n'))
    microbiome_lines_indices =microbiome_lines.zipWithIndex().map(lambda x: (x[1],x[0]))
    microbiome_characters = microbiome_lines_indices.filter(lambda x: x[0]%4 == 1).map(lambda x: separate(x[1]))
    microbiome_mapped_reduced = microbiome_characters.map(map_char).reduceByKey(lambda x, y: x + y)
    total_sum = microbiome_mapped_reduced.map(lambda x: x[1]).sum()
    g = microbiome_mapped_reduced.filter(lambda x: x[0] == "G").filter(lambda x: x[1]).take(1)[0][1]
    c = microbiome_mapped_reduced.filter(lambda x: x[0] == "C").filter(lambda x: x[1]).take(1)[0][1]
    GC_percentage = ((g+c)/total_sum)*100
    end = datetime.datetime.now()
    total_time = end - start
    print(f'GC content in the file "{filename}" is {GC_percentage:0.2f}%')
    print(f'Total runtime: {total_time}')

In [4]:
GC_content_count("hdfs://192.168.1.153:9000/team08/SRS055118.tar.bz2", "SRS055118.tar.bz2")

GC content in the file "SRS055118.tar.bz2" is 89.90%
Total runtime: 0:03:24.270642


In [5]:
GC_content_count("hdfs://192.168.1.153:9000/team08/joined.tar.bz2", "joined.tar.bz2")

GC content in the file "joined.tar.bz2" is 76.83%
Total runtime: 0:03:10.090163
